In [36]:
import numpy as np
import openrouteservice
from openrouteservice.geocode import pelias_search
import xarray as xr
from datetime import datetime
from datetime import time as dtime
from datetime import date as ddate

ORS_API_KEY = "5b3ce3597851110001cf62485c01b629a61e4f57b5449ab3a0000ec5"

def get_temp_series(location = "Bergheimer Straße 116, 69115 Heidelberg, Germany"):
    """
    Given a location returns an xarray dataset of hourly temperatures in Kelvin
    INPUTS:
        something
    OUTPUTS:
        ds (xarray dataset): has coordinates [longitude (float32), latitude (float32), time (datetime64)]
        
    TODO: specify time bounds, properly comment
    """
    clnt = openrouteservice.Client(key=ORS_API_KEY)
    call = pelias_search(clnt, location)
    for feature in call["features"]:
        geom = feature["geometry"]["coordinates"]

    fn = "temp_data.nc"
    ds = xr.open_dataset(fn)
    #yearmonth = "2021-10-31"
    #yearmonth = datetime.strptime(yearmonth, '%Y-%m-%d')
    ds = ds.sel(latitude=round(geom[1])).sel(longitude=round(geom[0]))

    # interpolate missing hours of the day (sometimes data only from 06h to 22h)
    ds = ds.resample(time="1h").interpolate("linear")
    return ds


def mask_temp(data, Tref = 291.15):
    """
    INPUTS:
        data (xarray dataset): Must have datetimes in ["time"].values and temperatures in ["t2m"].values
        Tref (float): Reference temperature, given in Kelvin. Default is 291.15 (18 Celcius).
    OUTPUTS:
        something
    Note that for now I choose not to use heating start and stop times (ex. setting temp = Tref outside this range)
    becuase it adds complications, ex. the building will maintain some of the heating even if it is off... I think
    that turning the heating off will result in lower gas usage anyway... idk should talk about it
    """
    pass

    # hourly time series
    # compute degree days

def get_month_endpoints(date_str):
    """
    Given a year-month string, return the endpoints required for the timeslice to select temperatures of a given month
    INPUTS:
        date_str (str): should be 'year-month', ex. '2020-02'
    OUTPUTS:
        start_str (str): string for first day of month, ex. '2020-02-01'
        end_str (str): string for first day of next month at midnight, so as to include 11pm to midnight of 
                        last day, ex. '2020-03-01-00'
    """
    start_year , start_month = date_str.split('-')
    if int(start_month) < 12:
        end_month = '{:02d}'.format(int(start_month)+1)
        end_year = start_year
    elif int(start_month) == 12:
        end_month = '01'
        end_year = str(int(start_year)+1)
    else:
        raise Exception('Error in get_month_endpoints: month can not be >12!')

    start_str = start_year + '-' + start_month + '-01'
    end_str = end_year + '-' + end_month + '-01-00'
    return start_str, end_str

def calc_degreedays(type, date_str, Tref=291.15):
    """ Docstring TODO """
    if type not in ('heating','cooling'):
        print("WARNING: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'
    
    ds = get_temp_series() 
    
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
    diff_time = month_ds['time'][-1] - month_ds['time'][0] # get time difference, saved as xarray with np.timedelta64 array in values

    if type == 'heating':
        month_ds['t2m'] = Tref - month_ds['t2m']
    else: # type == cooling
        month_ds['t2m'] = month_ds['t2m'] - Tref
    month_ds = np.maximum(month_ds, 0)

    # integrate
    # note: integrate sets each hour as unit 1, must divide by total number of hours to obtain cumulative degree days
    dd = month_ds.integrate('time',datetime_unit='h') / np.timedelta64(diff_time.values,'h').astype(float)
    
    return dd['t2m'].values

In [37]:
ds = get_temp_series()
#month_ds = ds.sel(time=slice(get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
#diff_time = month_ds['time'][-1] - month_ds['time'][0] # get time difference, saved as xarray with np.timedelta64 array in values
calc_degreedays('heating','2020-02')

array(12.00697915)

TODO:
- can we do calculation for each month simultaneously for a given total range?
- look into only calculating degree days based on fixed hours of the day
- look into alternative methods of calculating degree days, ex. soft cutoff below Tref or thresholding to a few degrees below Tref


1. Make it work with arrays. If we want for example to select the degree days for an entire year, ideally we would run this calculation simultaneously in an array for each month instead of looping over everything.

2. Select daily hourly ranges. As you say, you want to look into only calculating degree days based on fixed hours of the day. So we (a) have to think about the implications of this and how this affects the meaning of degree days, and (b) modify the code to only integrate between those ranges of the day

3. Look into alternative methods of calculating degree days. For example, we can have a soft cutoff below the reference temperatre instead of just cutting it to 0 as we do now. Or also for example as Veit was saying, we can thresholding to a few degrees below the reference temperature so the degree days kick in once it is a few degrees colder than the reference temperature

Thoughts:

1. 